In [ ]:
import karman

In [2]:
dataset=karman.ThermosphericDensityDataset(lag_minutes_omni=10, lag_days_fism2_daily=10, lag_minutes_fism2_flare=150)

Loading Thermospheric Density Dataset:
Loading OMNIWeb (1min) Dataset:
Loading FISM2 Daily Irradiance Dataset:
Loading FISM2 Flare (10min) Irradiance Dataset:


100%|██████████| 9/9 [00:02<00:00,  4.09it/s]


In [ ]:
import torch
from torch.utils.data.sampler import SubsetRandomSampler

with open('/home/jupyter/karman-project/scripts/train_indices.txt') as f:
    train_indices = [int(line.rstrip()) for line in f]
with open('/home/jupyter/karman-project/scripts/val_indices.txt') as f:
    val_indices = [int(line.rstrip()) for line in f]
with open('/home/jupyter/karman-project/scripts/test_indices.txt') as f:
    test_indices = [int(line.rstrip()) for line in f]

print(f"Train set proportion: {len(train_indices)/len(dataset)*100} %")
print(f"Validation set proportion: {len(val_indices)/len(dataset)*100} %")
print(f"Test set proportion: {len(test_indices)/len(dataset)*100} %")

#I perform the dataset split, creating train, valid, test dataloaders:
train_sampler=SubsetRandomSampler(train_indices)
valid_sampler=SubsetRandomSampler(val_indices)
test_sampler=SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(dataset,
                                       batch_size=1024,
                                       sampler=train_sampler,
                                       pin_memory=True,
                                       num_workers=16)
valid_loader = torch.utils.data.DataLoader(dataset,
                                       batch_size=1024,
                                       sampler=valid_sampler,
                                       pin_memory=True,
                                       num_workers=16)
test_loader = torch.utils.data.DataLoader(dataset,
                                       batch_size=1024,
                                       sampler=test_sampler,
                                       pin_memory=True,
                                       num_workers=16)

Train set proportion: 66.21214440424085 %
Validation set proportion: 16.421256649841354 %
Test set proportion: 17.3665989459178 %


In [17]:
item=next(iter(train_loader))

In [7]:
from karman import FFNN
from torch import optim
optimizer=optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.)
model = FFNN(num_features=len(dataset[0][0]))


In [19]:
inp,targ=item

model(inp).shape

torch.Size([1024, 1])